## Farmer problem 
### Victor M. Zavala 2016 
### University of Wisconsin-Madison 

In [79]:
using JuMP 
using Ipopt 

In [80]:
# Model parameters

NS = 5;                    # number of scenarios
S = collect([1:NS]);       # scenario set
P = collect([1:3]);        # set of crops (1=wheat,2=corn,3=beets)

# Data 
prcost = zeros(3)          # production (planting) cost
prcost[1] = 150;
prcost[2] = 230;
prcost[3] = 260;

pcost = zeros(3)           # purchase cost
pcost[1] = 238;
pcost[2] = 210;
pcost[3] = 0;

scost = zeros(3)           # sales cost
scost[1] = 170;
scost[2] = 150;
scost[3] = 36;

demand = zeros(3)          # demand
demand[1] = 200;
demand[2] = 240;
demand[3] = 0;

# assign random data

yield = zeros(length(S),3) # yields
yield[S,1] = 2.5;
yield[S,2] = 3.0;

yield[1,3] = 10;
yield[2,3] = 15;
yield[3,3] = 20;
yield[4,3] = 25;
yield[5,3] = 30;

# set cvar parameter

alpha = 5/5;              # probability level cvar

1.0

In [81]:
# Model 

m = Model(solver=IpoptSolver(print_level=0))

@defVar(m, x[S,P] >= 0)    # acres devoted to crops
@defVar(m, y[S,P] >= 0)    # crops purchase
@defVar(m, w[S,P] >= 0)    # crops sold;
@defVar(m, phi[S] >= 0)    # cvar auxiliary variable
@defVar(m, VaR)            # cvar auxiliary variable
@defExpr(m, Cost[s in S], sum{prcost[j]*x[s,j] + pcost[j]*y[s,j] - scost[j]*w[s,j], j in P})
@defVar(m, cost[s in S])

@addConstraint(m, varcost[s in S], cost[s] == Cost[s]) 
@addConstraint(m, cap[s in S], sum{x[s,j], j in P} <= 500)
@addConstraint(m, bal[s in S,j in P], yield[s,j]*x[s,j]+y[s,j]-w[s,j] >= demand[j])
@addConstraint(m, sellb[s in S], w[s,3] <= 6000)
@addConstraint(m, buyb[s in S], y[s,3] <= 0)
@addConstraint(m, nonant[s in S,j in P], x[1,j] == x[s,j])
@addConstraint(m, cvar[s in S], cost[s]-VaR <= phi[s])

@setObjective(m, Min, VaR + (1/NS)*sum{(1/alpha)*phi[s], s in S})
 
solve(m)

:Optimal

In [82]:
# Results

println(getValue(x))
println("")
println(getValue(y))
println("")
println(getValue(w))
println("")
println(getValue(cost))
println("")
println("obj: ", getObjectiveValue(m))

x: 2 dimensions:
[1,:]
  [1,1] = 220.00000379973474
  [1,2] = 79.99999920015239
  [1,3] = 200.00000200006727
[2,:]
  [2,1] = 220.00000379973474
  [2,2] = 79.99999920015239
  [2,3] = 200.00000200006727
[3,:]
  [3,1] = 220.00000379973474
  [3,2] = 79.99999920015239
  [3,3] = 200.00000200006727
[4,:]
  [4,1] = 220.00000379973474
  [4,2] = 79.99999920015239
  [4,3] = 200.00000200006727
[5,:]
  [5,1] = 220.00000379973474
  [5,2] = 79.99999920015239
  [5,3] = 200.00000200006727


y: 2 dimensions:
[1,:]
  [1,1] = 0.0
  [1,2] = 0.0
  [1,3] = 0.0
[2,:]
  [2,1] = 0.0
  [2,2] = 0.0
  [2,3] = 0.0
[3,:]
  [3,1] = 0.0
  [3,2] = 0.0
  [3,3] = 0.0
[4,:]
  [4,1] = 0.0
  [4,2] = 0.0
  [4,3] = 0.0
[5,:]
  [5,1] = 0.0
  [5,2] = 0.0
  [5,3] = 0.0


w: 2 dimensions:
[1,:]
  [1,1] = 350.0000114894474
  [1,2] = 0.0
  [1,3] = 2000.000020000325
[2,:]
  [2,1] = 350.0000114894474
  [2,2] = 0.0
  [2,3] = 3000.000030000662
[3,:]
  [3,1] = 350.0000114894474
  [3,2] = 0.0
  [3,3] = 4000.000040000998
[4,:]
  [4,1] = 3